In [1]:
import random
import itertools
import collections
import time

In [2]:
class Node:
    """
    A class representing an Solver node
    - 'puzzle' is a Puzzle instance
    - 'parent' is the preceding node generated by the solver, if any
    - 'action' is the action taken to produce puzzle, if any
    """
    def __init__(self, puzzle, parent=None, action=None):
        self.puzzle = puzzle
        self.parent = parent
        self.action = action
        if (self.parent != None):
            self.g = parent.g + 1
        else:
            self.g = 0

    @property
    def score(self):
        return (self.g + self.h)

    @property
    def state(self):
        """
        Return a hashable representation of self
        """
        return str(self)

    @property 
    def path(self):
        """
        Reconstruct a path from to the root 'parent'
        """
        node, p = self, []
        while node:
            p.append(node)
            node = node.parent
        yield from reversed(p)

    @property
    def solved(self):
        """ Wrapper to check if 'puzzle' is solved """
        return self.puzzle.solved

    @property
    def actions(self):
        """ Wrapper for 'actions' accessible at current state """
        return self.puzzle.actions

    @property
    def h(self):
        """"h"""
        return self.puzzle.manhattan

    @property
    def f(self):
        """"f"""
        return self.h + self.g

    def __str__(self):
        return str(self.puzzle)


In [3]:
class Solver:
    """
    An '8-puzzle' solver
    - 'start' is a Puzzle instance
    """
    def __init__(self, start):
        self.start = start

    def solve(self):
        """
        Perform breadth first search and return a path
        to the solution, if it exists
        """
        queue = collections.deque([Node(self.start)])
        seen = set()
        seen.add(queue[0].state)
        while queue:
            queue = collections.deque(sorted(list(queue), key=lambda node: node.f))
            node = queue.popleft()
            if node.solved:
                return node.path

            for move, action in node.actions:
                child = Node(move(), node, action)

                if child.state not in seen:
                    queue.appendleft(child)
                    seen.add(child.state)

In [4]:
class Puzzle:
    """
    A class representing an '8-puzzle'.
    - 'board' should be a square list of lists with integer entries 0...width^2 - 1
       e.g. [[1,2,3],[4,0,6],[7,5,8]]
    """
    def __init__(self, board):
        self.width = len(board[0])
        self.board = board

    @property
    def solved(self):
        """
        The puzzle is solved if the flattened board's numbers are in
        increasing order from left to right and the '0' tile is in the
        last position on the board
        """
        N = self.width * self.width
        return str(self) == ''.join(map(str, range(1,N))) + '0'

    @property 
    def actions(self):
        """
        Return a list of 'move', 'action' pairs. 'move' can be called
        to return a new puzzle that results in sliding the '0' tile in
        the direction of 'action'.
        """
        def create_move(at, to):
            return lambda: self._move(at, to)

        moves = []
        for i, j in itertools.product(range(self.width),
                                      range(self.width)):
            direcs = {'R':(i, j-1),
                      'L':(i, j+1),
                      'D':(i-1, j),
                      'U':(i+1, j)}

            for action, (r, c) in direcs.items():
                if r >= 0 and c >= 0 and r < self.width and c < self.width and \
                   self.board[r][c] == 0:
                    move = create_move((i,j), (r,c)), action
                    moves.append(move)
        return moves

    @property
    def manhattan(self):
        distance = 0
        for i in range(3):
            for j in range(3):
                if self.board[i][j] != 0:
                    x, y = divmod(self.board[i][j]-1, 3)
                    distance += abs(x - i) + abs(y - j)
        return distance

    def shuffle(self):
        """
        Return a new puzzle that has been shuffled with 1000 random moves
        """
        puzzle = self
        for _ in range(1000):
            puzzle = random.choice(puzzle.actions)[0]()
        return puzzle

    def copy(self):
        """
        Return a new puzzle with the same board as 'self'
        """
        board = []
        for row in self.board:
            board.append([x for x in row])
        return Puzzle(board)

    def _move(self, at, to):
        """
        Return a new puzzle where 'at' and 'to' tiles have been swapped.
        NOTE: all moves should be 'actions' that have been executed
        """
        copy = self.copy()
        i, j = at
        r, c = to
        copy.board[i][j], copy.board[r][c] = copy.board[r][c], copy.board[i][j]
        return copy

    def pprint(self):
        for row in self.board:
            print(row)
        print()

    def __str__(self):
        return ''.join(map(str, self))

    def __iter__(self):
        for row in self.board:
            yield from row

In [85]:
# example of use     
board = [[4,5,1],[8,3,7],[6,2,0]]
#random.shuffle(board)
puzzle = Puzzle(board)
puzzle = puzzle.shuffle()
s = Solver(puzzle)
tic = time.clock()
p = s.solve()
toc = time.clock()

steps = 0
for node in p:
  #print(node.action)
  print("Step: " + str(steps))
  node.puzzle.pprint()
  steps += 1
#else:
  #print("No solution!")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  import sys


Step: 0
[5, 3, 2]
[7, 0, 6]
[4, 8, 1]

Step: 1
[5, 3, 2]
[0, 7, 6]
[4, 8, 1]

Step: 2
[0, 3, 2]
[5, 7, 6]
[4, 8, 1]

Step: 3
[3, 0, 2]
[5, 7, 6]
[4, 8, 1]

Step: 4
[3, 2, 0]
[5, 7, 6]
[4, 8, 1]

Step: 5
[3, 2, 6]
[5, 7, 0]
[4, 8, 1]

Step: 6
[3, 2, 6]
[5, 7, 1]
[4, 8, 0]

Step: 7
[3, 2, 6]
[5, 7, 1]
[4, 0, 8]

Step: 8
[3, 2, 6]
[5, 0, 1]
[4, 7, 8]

Step: 9
[3, 0, 6]
[5, 2, 1]
[4, 7, 8]

Step: 10
[0, 3, 6]
[5, 2, 1]
[4, 7, 8]

Step: 11
[5, 3, 6]
[0, 2, 1]
[4, 7, 8]

Step: 12
[5, 3, 6]
[2, 0, 1]
[4, 7, 8]

Step: 13
[5, 3, 6]
[2, 1, 0]
[4, 7, 8]

Step: 14
[5, 3, 0]
[2, 1, 6]
[4, 7, 8]

Step: 15
[5, 0, 3]
[2, 1, 6]
[4, 7, 8]

Step: 16
[5, 1, 3]
[2, 0, 6]
[4, 7, 8]

Step: 17
[5, 1, 3]
[0, 2, 6]
[4, 7, 8]

Step: 18
[0, 1, 3]
[5, 2, 6]
[4, 7, 8]

Step: 19
[1, 0, 3]
[5, 2, 6]
[4, 7, 8]

Step: 20
[1, 2, 3]
[5, 0, 6]
[4, 7, 8]

Step: 21
[1, 2, 3]
[0, 5, 6]
[4, 7, 8]

Step: 22
[1, 2, 3]
[4, 5, 6]
[0, 7, 8]

Step: 23
[1, 2, 3]
[4, 5, 6]
[7, 0, 8]

Step: 24
[1, 2, 3]
[4, 5, 6]
[7, 8, 0]



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  if __name__ == '__main__':


In [86]:
print("Total number of steps: " + str(steps))
print("Total amount of time in search: " + str(toc - tic) + " second(s)")

Total number of steps: 25
Total amount of time in search: 18.4299299999999 second(s)
